In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.data import Field

from dataset.mtgcards import RuleText
from utils.preprocess import fields_for_rule_text

import random
from tqdm import tqdm

import models.model6_1 as model6_1
from dataset.mtgcards import TestSets
from models.card_name_detector.definition import TrainedDetector
from utils.translate import sentencize, CardTranslator, CTHelper

d:\Miniconda3\envs\seq2seq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fields = {'src-rule': ('src', Field(tokenize=lambda x: x.split(' '))), 'trg-rule': ('trg', Field())}
test_data = TestSets.load(fields)

In [3]:
T = {
    'model4-T-v2.1': torch.load('result/model4-T-v2.1.pt'),
    'model4-T-v2.2': torch.load('result/model4-T-v2.2.pt'),
    #'model6-T-v2.1': torch.load('result/model6-T-v2.1.pt'),
    'model6-T-v2.2': torch.load('result/model6-T-v2.2.pt'),
    'model6.1-T-v2.2': torch.load('result/model6.1-T-v2.2.pt'),
    #'hybrid-T-v2.2': torch.load('result/hybrid-T-v2.2.pt'),
}
D = TrainedDetector()

path: d:\Desktop\mtg-cards-translation\models\card_name_detector


In [4]:
dic = {}
#dic = {'prototype':'试作','powerstone':'魔力石'}
dic = {'oil':'烁油', 'rebel':'反抗军','compleat':'完化'}
helper = CTHelper(D, dic)
silent = True
CT={}
for key in T:
    value = T[key]
    CT[key]=CardTranslator(sentencize, value, 
                    preprocess=lambda x: helper.preprocess(x, silent), 
                    postprocess=lambda x: helper.postprocess(x, silent))

# ret = CT.translate(' '.join(example.src))
# print(ret)
from utils import calculate_testset_bleu
for example in random.sample(list(test_data), 1):
    #example=list(test_data)[13]
    for key in CT:
        ct=CT[key]
        print(vars(example))
        ret = ct.translate(' '.join(example.src))
        print(ret)
        #print(calculate_testset_bleu(list(test_data)[:100], ct))



{'src': ['Whenever', 'Escaped', 'Experiment', 'attacks,', 'target', 'creature', 'an', 'opponent', 'controls', 'gets', '-X/-0', 'until', 'end', 'of', 'turn,', 'where', 'X', 'is', 'the', 'number', 'of', 'artifacts', 'you', 'control.'], 'trg': ['每当逃脱实验体攻击时，目标由对手操控的生物得-X/-0直到回合结束，X为由你操控的神器数量。']}
每当<escaped experiment>攻击时，目标由对手操控的生物得-x/-0直到回合结束，x为由你操控的神器数量。
{'src': ['Whenever', 'Escaped', 'Experiment', 'attacks,', 'target', 'creature', 'an', 'opponent', 'controls', 'gets', '-X/-0', 'until', 'end', 'of', 'turn,', 'where', 'X', 'is', 'the', 'number', 'of', 'artifacts', 'you', 'control.'], 'trg': ['每当逃脱实验体攻击时，目标由对手操控的生物得-X/-0直到回合结束，X为由你操控的神器数量。']}
每当<escaped experiment>攻击时，目标由对手操控的生物得-x/-0直到回合结束，x为由你操控的神器数量。
{'src': ['Whenever', 'Escaped', 'Experiment', 'attacks,', 'target', 'creature', 'an', 'opponent', 'controls', 'gets', '-X/-0', 'until', 'end', 'of', 'turn,', 'where', 'X', 'is', 'the', 'number', 'of', 'artifacts', 'you', 'control.'], 'trg': ['每当逃脱实验体攻击时，目标由对手操控的生物得-X/-0直到回合结束，X为由你操控的神器数量。'